In [1]:
import pandas as pd
from sqlalchemy import create_engine
import py
import pymysql
from config import gkey
import requests
import json
from pprint import pprint

# Yelp Data

In [2]:
yelp_json = 'Yelp_Data/yelp_business.json'

#adding the commas to the json file
with open (yelp_json,'r', encoding='utf-8') as cmnts:
    all_entries = cmnts.readlines()
    
parsable_data = "[" + ','.join(all_entries).replace('\n','') + "]"
yelp_df = pd.read_json(parsable_data, encoding = 'utf-8')
yelp_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,None,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB


Cleaning Data

In [3]:
new_yelp_df = yelp_df[['name','categories','address','city','state','postal_code','review_count','stars' ]].copy()

new_yelp_df
                      


,name,categories,address,city,state,postal_code,review_count,stars
0,Minhas Micro Brewery,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",1314 44 Avenue NE,Calgary,AB,T2E 6L6,24,4.0
1,CK'S BBQ & Catering,"Chicken Wings, Burgers, Caterers, Street Vendo...",,Henderson,NV,89002,3,4.5
2,La Bastringue,"Breakfast & Brunch, Restaurants, French, Sandw...",1335 rue Beaubien E,Montréal,QC,H2G 1K7,5,4.0
3,Geico Insurance,"Insurance, Financial Services",211 W Monroe St,Phoenix,AZ,85003,8,1.5
4,Action Engine,"Home & Garden, Nurseries & Gardening, Shopping...",2005 Alyth Place SE,Calgary,AB,T2H 0N5,4,2.0
5,The Coffee Bean & Tea Leaf,"Coffee & Tea, Food","20235 N Cave Creek Rd, Ste 1115",Phoenix,AZ,85024,63,4.0
6,Bnc Cake House,"Food, Bakeries",631 Bloor St W,Toronto,ON,M6G 1K8,7,4.0
7,Thai One On,"Restaurants, Thai","3417 Derry Road E, Unit 103",Mississauga,ON,L4T 1A8,7,2.0
8,Filiberto's Mexican Food,"Mexican, Restaurants",1440 N. Dysart Ave,Avondale,AZ,85323,40,2.5
9,Maggie & Stella's Gifts,"Flowers & Gifts, Gift Shops, Shopping",209 Oakland Ave,Pittsburgh,PA,15213,3,3.5


In [4]:
pits_yelp_df = new_yelp_df.loc[new_yelp_df['city']=='Pittsburgh']
pits_yelp_df

,name,categories,address,city,state,postal_code,review_count,stars
9,Maggie & Stella's Gifts,"Flowers & Gifts, Gift Shops, Shopping",209 Oakland Ave,Pittsburgh,PA,15213,3,3.5
67,Philip Pelusi,"Hair Salons, Beauty & Spas","531 Clairton Blvd, Unit 7",Pittsburgh,PA,15236,5,4.5
68,FedEx Office Print & Ship Center,"Professional Services, Local Services, Shippin...",1720 Washington Rd,Pittsburgh,PA,15241,5,3.5
73,China Palace,"Asian Fusion, Caterers, Fast Food, Chinese, Re...",5440 Walnut St,Pittsburgh,PA,15232,110,3.0
89,Pizza Bellagio,"Pizza, Restaurants",4635 Centre Ave,Pittsburgh,PA,15213,18,1.5
189,The Gilded Girl,"Beauty & Spas, Shopping, Specialty Schools, Ed...",5104 Butler St,Pittsburgh,PA,15201,11,4.5
191,Evolve Modern Hairdressing,"Beauty & Spas, Hair Salons",4900 Hatfield St,Pittsburgh,PA,15201,13,4.0
217,The Diamond Group Home Inspections,"Home Inspectors, Home Services",,Pittsburgh,PA,15222,3,5.0
234,Commonwealth Press,"Local Services, Screen Printing",1931 E Carson St,Pittsburgh,PA,15203,15,4.5
236,iCracked iPhone Repair,"IT Services & Computer Repair, Local Services,...",,Pittsburgh,PA,15203,7,4.5


In [5]:
pits_yelp_df = pits_yelp_df.dropna()
pits_yelp_df = pits_yelp_df[pits_yelp_df['categories'].str.contains('Restaurants')]


In [6]:
#encode name column to UTF 8
pits_yelp_df.name = pits_yelp_df.name.str.encode('utf-8')

In [16]:
rds_connection_string = "root:shazam@127.0.0.1/restaurant_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [8]:
#export to Mysql
pits_yelp_df.to_sql('yelp_reviews', con = engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

# Yellow Page Data

In [123]:
csv_file = "Resources/yellowpages.csv"

yellowpagesraw_df = pd.read_csv(csv_file)
yellowpagesraw_df.head()

,Uniq Id,Url,Name,Street,Zip Code,City,State,Phone,Email,Website,Categories,Rating
0,85c7c0ebada10ae36b0ffe6e72e92ed5,http://www.yellowpages.com/indianapolis-in/mip...,454,7280 Michigan Rd,46268.0,Indianapolis,IN,(317) 298-9001,NaN,NaN,"Continental Restaurants, Restaurants",1.0
1,e81c3082aafaf4e57883140616c516bd,http://www.yellowpages.com/philadelphia-pa/mip...,10 Arts,10 Avenue Of The Arts # Lobby,19102.0,Philadelphia,PA,(215) 523-8273,phila.10arts@ritzcarlton.com,http://www.10arts.com,"Continental Restaurants, French Restaurants, R...",1.5
2,92442b01e2f29fd1d8eef92ba2f91f09,http://www.yellowpages.com/miami-fl/mip/100-mo...,100 Montaditos,244 Biscayne Blvd,33132.0,Miami,FL,(786) 347-3065,NaN,http://www.100montaditos.com,"Spanish Restaurants, Restaurants, Sandwich Shops",2.0
3,f6d2b4e33a6c29860f34ef0217f33b97,http://www.yellowpages.com/indianapolis-in/mip...,120 West Market Fresh Grill,120 W Market St,46204.0,Indianapolis,IN,(317) 822-5824,NaN,NaN,Restaurants,2.5
4,0a78c2aa7744435ce55edde64c9254b6,http://www.yellowpages.com/philadelphia-pa/mip...,13th St Pizza,209 S 13th St,19107.0,Philadelphia,PA,(215) 546-4453,NaN,NaN,"Pizza, American Restaurants, Italian Restauran...",3.0


In [122]:
yellowpagespitt_df = yellowpagesraw_df.loc[yellowpagesraw_df["City"]=="Pittsburgh"]
yellowpagespitt_df.head()

,Uniq Id,Url,Name,Street,Zip Code,City,State,Phone,Email,Website,Categories,Rating
25,c0dea87ba09e8f82da541963b09a2b72,http://www.yellowpages.com/pittsburgh-pa/mip/5...,5 Fools,9566 Perry Hwy,15237.0,Pittsburgh,PA,(412) 366-3544,NaN,NaN,Restaurants,4.5
45,82f675b150359cf9b7153ba964a963f2,http://www.yellowpages.com/pittsburgh-pa/mip/a...,A&M Gyros,3536 Saw Mill Run Blvd,15227.0,Pittsburgh,PA,(412) 885-1010,NaN,NaN,"Greek Restaurants, Fast Food Restaurants, Medi...",1.0
162,b4cfe5fd4413d7fee6142bec935e8c80,http://www.yellowpages.com/pittsburgh-pa/mip/a...,Anthony's,950 Ohio River Blvd,15202.0,Pittsburgh,PA,(412) 734-4000,KillenR@astampcoin.com,http://anthonysfla.com,"Caterers, Restaurants",1.0
169,65bf7d01bb7fba7337e6efc13c84c4ff,http://www.yellowpages.com/pittsburgh-pa/mip/a...,Antonio's Pizzeria,758 Brookline Blvd,15226.0,Pittsburgh,PA,(412) 388-1000,NaN,http://antonios-pizza.com,"Pizza, Restaurants",4.5
448,0256d1b01ca051b457ab423a66cf8416,http://www.yellowpages.com/pittsburgh-pa/mip/b...,BLIMPIE,307 Smithfield St,15222.0,Pittsburgh,PA,(412) 281-0099,NaN,http://www.blimpie.com,"Delicatessens, Take Out Restaurants, Sandwich ...",4.5


In [120]:
yppitt_df = yellowpagespitt_df[["Name", "Street", "City","State", "Rating"]]
yppitt_df.head()

,Name,Street,City,State,Rating
25,5 Fools,9566 Perry Hwy,Pittsburgh,PA,4.5
45,A&M Gyros,3536 Saw Mill Run Blvd,Pittsburgh,PA,1.0
162,Anthony's,950 Ohio River Blvd,Pittsburgh,PA,1.0
169,Antonio's Pizzeria,758 Brookline Blvd,Pittsburgh,PA,4.5
448,BLIMPIE,307 Smithfield St,Pittsburgh,PA,4.5


In [121]:
YP_df = yppitt_df.rename(columns={"Name":"Restaurant_YP", "Street":"address", "City":"City_YP","State":"State_YP", "Rating":"Rating_YP"})
YP_df.head()

,Restaurant_YP,address,City_YP,State_YP,Rating_YP
25,5 Fools,9566 Perry Hwy,Pittsburgh,PA,4.5
45,A&M Gyros,3536 Saw Mill Run Blvd,Pittsburgh,PA,1.0
162,Anthony's,950 Ohio River Blvd,Pittsburgh,PA,1.0
169,Antonio's Pizzeria,758 Brookline Blvd,Pittsburgh,PA,4.5
448,BLIMPIE,307 Smithfield St,Pittsburgh,PA,4.5


In [126]:
YP_df.to_sql('yp_reviews', con = engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

# Google Data

In [9]:
target_city = "Pittsburgh, Pennsylvania"

params = {"address": target_city, "key": gkey}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# Run request
response = requests.get(base_url, params=params)

# Convert to JSON
jc_geo = response.json()

# Extract lat/lng
lat = jc_geo["results"][0]["geometry"]["location"]["lat"]
lng = jc_geo["results"][0]["geometry"]["location"]["lng"]

# Print results
print(f"{target_city}: {lat}, {lng}")

Pittsburgh, Pennsylvania: 40.44062479999999, -79.9958864


In [10]:
# geocoordinates
target_coordinates = "40.44062479999999, -79.9958864"
target_radius = 1000
target_type = "restaurant"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
pitt_data = response.json()

# Print the json (pretty printed)
print(json.dumps(pitt_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "next_page_token": "CtQETQIAAESqeyoH2l4G7ZRoWlISE3Wu4xcDiBOqKHV5MiOoM3P0CIxaJ2sCHQdKXwUR5HGOYaP1TLbDAfzwI0tHtDj_coITgr9x1sLQ0IS5y_5G3jnK_si32ffg7X2Ea6H9yTsEehxn93eygIZxFhw4PoeClZKwSgvR7gla3w-I0aD0X6cp_oo0ReYZXOCfzvxV-DDO7UUEp9mQKEf7RdNpvnXY-cnVayvQVOyUTyrhMGk7Qo5qo6cH5zelkE1XBUPnFV53zG59oI26iur9vKIUw7YBnXJGSYZCSjYBU_FKxYwVnoBNe8o6z8zrYOHu_X-Yh05_5canPK6NkiKxdBKWxUBuErBDki7ANZNkFDi4LYhJAgxFjOreaF3ERBK0CTt2YLNQqixifhD3W0c1x5K9laBWyVt0yuNSXXeWjLkITdrGLHosmJJLZrl76RHV5YmGne3MMVhyzjCSAsLfCMO2E4fTyyBHjPqaCxr-BV6KivF8bM8JwT-uU2rRVQG_4hhPI_6NCYkv4KmvrClurlp57Qh5zPT6RMgteo93Y64Uf_lJ0scpH1AxNkgZG8F8UgoVV-8KM3a8CE_K_ukf5zFiZ3WWrcrnhv48X7XMk8JAE6mpaUSzmTkoQdNAD6zAcMi2DbeoT0J3U0esUfAelOcHcz7x_dGQlqDdWuGB6voRAH1-EsgVL-UQbpTxpEL2LuTLdMTrpRsl3Eqe_V4xPQmCuwOGFLwNTHuWWGujJbIaPNiHMryk3SNRsMdC6nL5wwazEjVESg7kqcxnA-ASqTtW6PmtRUkSEFZ_v6zvZAfroQzNOJ_wxJYaFG6kT38rSrFG9rT0VjR00j8XvxuT",
    "results": [
        {
            "geometry": {
                "location": {
       

In [11]:
pprint(pitt_data["results"])
pitt_list = pitt_data["results"]

[{'geometry': {'location': {'lat': 40.44315030000001, 'lng': -80.0011895},
               'viewport': {'northeast': {'lat': 40.44443038029149,
                                          'lng': -79.9998085197085},
                            'southwest': {'lat': 40.44173241970849,
                                          'lng': -80.0025064802915}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
  'id': 'e4853e16b92352af2db90e7606628d66b541a725',
  'name': 'Meat & Potatoes',
  'opening_hours': {'open_now': False},
  'photos': [{'height': 626,
              'html_attributions': ['<a '
                                    'href="https://maps.google.com/maps/contrib/101632382784807640089/photos">Meat '
                                    '&amp; Potatoes</a>'],
              'photo_reference': 'CmRaAAAAb3XFCNLkbDV_KcmEGcmgy9roeZrVy9GJVekwKhNWaQTb9KJPfqDKoy6wRb73UCoVHEE0snt_VcaPMDgtW2aSg9mOEIOYxMYZ6N5IrmQgniN9_FjYDMkH_ZDARUSyzG5FEhAdBO0eDhV9A6FiYAX6xwFRGhSJ7T

In [12]:
pitt_list[0]

{'geometry': {'location': {'lat': 40.44315030000001, 'lng': -80.0011895},
  'viewport': {'northeast': {'lat': 40.44443038029149,
    'lng': -79.9998085197085},
   'southwest': {'lat': 40.44173241970849, 'lng': -80.0025064802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png',
 'id': 'e4853e16b92352af2db90e7606628d66b541a725',
 'name': 'Meat & Potatoes',
 'opening_hours': {'open_now': False},
 'photos': [{'height': 626,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101632382784807640089/photos">Meat &amp; Potatoes</a>'],
   'photo_reference': 'CmRaAAAAb3XFCNLkbDV_KcmEGcmgy9roeZrVy9GJVekwKhNWaQTb9KJPfqDKoy6wRb73UCoVHEE0snt_VcaPMDgtW2aSg9mOEIOYxMYZ6N5IrmQgniN9_FjYDMkH_ZDARUSyzG5FEhAdBO0eDhV9A6FiYAX6xwFRGhSJ7TaEIE5NpBrldjMyNkxFr9ZXcw',
   'width': 1252}],
 'place_id': 'ChIJ1QL2R1bxNIgR3zR9Fz4lorg',
 'plus_code': {'compound_code': 'CXVX+7G Pittsburgh, Pennsylvania, United States',
  'global_code': '86GXCXVX+7G'},
 'price_level': 3

In [13]:
name = []
address = []
rating = []

for restaurant in pitt_list:
   name.append(restaurant["name"])
   address.append(restaurant["vicinity"])
   rating.append(restaurant["rating"])

df = pd.DataFrame({"name":name,"vicinity":address,"rating":rating})
df

,name,vicinity,rating
0,Meat & Potatoes,"649 Penn Avenue, Pittsburgh",4.5
1,McCormick & Schmick's Seafood & Steaks,"301 Fifth Avenue, Pittsburgh",4.0
2,Olive Or Twist,"140 6th Street, Pittsburgh",4.1
3,The Carlton Restaurant,"500 Grant Street, Pittsburgh",4.5
4,Bigelow Grille,"1 Bigelow Square, Pittsburgh",4.1
5,The Capital Grille,"301 Fifth Avenue, Pittsburgh",4.6
6,Steelhead Brasserie and Wine Bar,"112 Washington Place, Pittsburgh",3.3
7,Braddock's Rebellion,"107 6th Street, Pittsburgh",4.1
8,Primanti Bros.,"2 Market Square, Pittsburgh",4.3
9,Mexico City,"409 Wood Street, Pittsburgh",3.4


In [14]:
LG_df = df.rename(columns={"name":"Restaurant_G", "vicinity":"Address_G", "rating":"Rating_G"})
LG_df


,Restaurant_G,Address_G,Rating_G
0,Meat & Potatoes,"649 Penn Avenue, Pittsburgh",4.5
1,McCormick & Schmick's Seafood & Steaks,"301 Fifth Avenue, Pittsburgh",4.0
2,Olive Or Twist,"140 6th Street, Pittsburgh",4.1
3,The Carlton Restaurant,"500 Grant Street, Pittsburgh",4.5
4,Bigelow Grille,"1 Bigelow Square, Pittsburgh",4.1
5,The Capital Grille,"301 Fifth Avenue, Pittsburgh",4.6
6,Steelhead Brasserie and Wine Bar,"112 Washington Place, Pittsburgh",3.3
7,Braddock's Rebellion,"107 6th Street, Pittsburgh",4.1
8,Primanti Bros.,"2 Market Square, Pittsburgh",4.3
9,Mexico City,"409 Wood Street, Pittsburgh",3.4


In [15]:
# new data frame with split value columns 
g_df = LG_df["Address_G"].str.split(", ", n = 2, expand = True) 
  
# making seperate first name column from new data frame 
LG_df["Address_G"]= g_df[0] 

LG_df = LG_df.rename(columns = {'Address_G': 'address'})
LG_df
  
# making seperate last name column from new data frame 
# data["Last Name"]= g_df[1] 
  
# Dropping old Name columns 
# data.drop(columns =["Name"], inplace = True)

,Restaurant_G,address,Rating_G
0,Meat & Potatoes,649 Penn Avenue,4.5
1,McCormick & Schmick's Seafood & Steaks,301 Fifth Avenue,4.0
2,Olive Or Twist,140 6th Street,4.1
3,The Carlton Restaurant,500 Grant Street,4.5
4,Bigelow Grille,1 Bigelow Square,4.1
5,The Capital Grille,301 Fifth Avenue,4.6
6,Steelhead Brasserie and Wine Bar,112 Washington Place,3.3
7,Braddock's Rebellion,107 6th Street,4.1
8,Primanti Bros.,2 Market Square,4.3
9,Mexico City,409 Wood Street,3.4


In [17]:
LG_df.to_sql('google_reviews', con = engine, schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)